## Decision Trees for Class Imbalance with Scikit-Learn

Kaggle CreditCard Fraud Detection Data can be downloaded here:
https://github.com/nsethi31/Kaggle-Data-Credit-Card-Fraud-Detection/blob/master/creditcard.csv?raw=true

### How Decision Trees work:

Part 1: https://www.youtube.com/watch?v=O__7lAqni7A
    
Part 2: https://www.youtube.com/watch?v=WpA-Xbw4z_A

In [12]:
%%time
import pandas as pd
data = pd.read_csv('creditcard.csv')

CPU times: user 2.2 s, sys: 324 ms, total: 2.52 s
Wall time: 6.04 s


In [13]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [14]:
data.shape

(284807, 31)

In [15]:
data['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [16]:
284315/492

577.8760162601626

In [17]:
%%time
# Split data into train and test splits

from sklearn.model_selection import train_test_split

# retrieve numpy array
data = data.values
# split into input and output elements
X, y = data[:, 1:-1], data[:, -1]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=4992)

CPU times: user 298 ms, sys: 75.2 ms, total: 373 ms
Wall time: 951 ms


In [18]:
# Count how many unique values of each class

import numpy as np
unique, counts = np.unique(y_train, return_counts=True)
print (np.asarray((unique, counts)).T)

unique, counts = np.unique(y_test, return_counts=True)
print (np.asarray((unique, counts)).T)

[[0.0000e+00 2.1324e+05]
 [1.0000e+00 3.6500e+02]]
[[0.0000e+00 7.1075e+04]
 [1.0000e+00 1.2700e+02]]


### The class weighing can be defined multiple ways; for example:

* Domain expertise, determined by talking to subject matter experts
* Tuning, determined by a hyperparameter search such as a grid search
* Heuristic, specified using a general best practice
* A best practice for using the class weighting is to use the inverse of the class distribution present in the training dataset

In [19]:
# calculate heuristic class weighting
from sklearn.utils.class_weight import compute_class_weight

# calculate class weighting according to training data
weighting = compute_class_weight('balanced', [0,1], y_train)
print(weighting)

[  0.50085584 292.60958904]


### For a decision tree:
* *Small Weight:* Less importance, lower impact on node purity.
* *Large Weight:* More importance, higher impact on node purity.

In [ ]:
%%time
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

weights = {0:weighting[0], 1:weighting[1]} 
print(weights)
# define model
# try with class_weight=weights and class_weight=None
model = DecisionTreeClassifier()
#model = DecisionTreeClassifier(class_weight=weights)
# fit a model
model.fit(X_train, y_train)

# predict probabilities
probs = model.predict_proba(X_test)
# keep probabilities for the positive outcome only
pos_probs = probs[:, 1]

auc = roc_auc_score(y_test, pos_probs)

# summarize performance
print(' ROC AUC = %.3f' % auc)

{0: 0.5008558431813919, 1: 292.6095890410959}


In [10]:
%%time
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

weights = [
        {0:weighting[0], 1:weighting[1]} ,
    {0:0.7, 1:300} ,
    {0:22, 1:105} ,
    {0:50, 1:150} 
          ]
for w in weights:
    # define model
    # try with class_weight=weights and class_weight=None
    #model = DecisionTreeClassifier()
    model = DecisionTreeClassifier(class_weight=w)
    # fit a model
    model.fit(X_train, y_train)

    # predict probabilities
    probs = model.predict_proba(X_test)
    # keep probabilities for the positive outcome only
    pos_probs = probs[:, 1]

    auc = roc_auc_score(y_test, pos_probs)

    # summarize performance
    print(' ROC AUC = %.3f' % auc)

 ROC AUC = 0.874
 ROC AUC = 0.865
 ROC AUC = 0.897
 ROC AUC = 0.905
CPU times: user 39.1 s, sys: 802 ms, total: 39.9 s
Wall time: 1min 25s
